# Image Analysis imgAna_OpenCV 

In the past a service was invoked to locate faces, this is a replacement for that outboard processing locating
the faces using an OpenCV components.

The exploration process of this can is found imageAnaExplore_OpenCV, which maybe like going to a
sausage factory. 


Two types of analysis are being done on the images. 
- FaceRegion() : finds faces, this what I used to learn on, thus all the work a the begining of the notebook.
- ObjectRegions() : finds objects, much better example since it give the location of the object and an key to what the object is. This I got from Jerome. 

Two applications are composed here.... 
- imageFetch : pulls images in from the web and encodes them
- fineImageElements : invokes FaceRegions() to find faces & ObjectRegions()

The images are flowing in via EventStreams(Kafka) from the VideoSndKafka.py a stand alone program that shreds
videos from the internet.archive.org pushes the across. Which means you need to have instantiated the EventStreams, I'm using the Cloud instance since I don't have the where-with-all to set it up. 





In [ ]:
#!pip install opencv-contrib-python
#%load_ext autoreload
#%autoreload 2

## Note : 

Getting this build locally and push up to the cloud you need the following...
```bash
pip install opencv-contrib-python
pip install pandas
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
pip install beautifulsoup4
pip install Pillow
```
Here is where I got a jump start on [face-detection-python-opencv](https://www.datacamp.com/community/tutorials/face-detection-python-opencv). 


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from IPython.core.debugger import set_trace
from IPython.display import display, clear_output

from statistics import mean
from collections import deque
from collections import Counter

import json
import datetime 

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout

from bs4 import BeautifulSoup

%matplotlib inline

from sseclient import SSEClient as EventSource

from ipywidgets import Button, HBox, VBox, Layout

from  functools import lru_cache
import requests

from streamsx.topology.topology import *
import streamsx.rest as rest
from streamsx.topology import context

import time

from PIL import Image,  ImageDraw  # https://pillow.readthedocs.io/en/4.3.x/
import io
import base64
import sys
if '../scripts' not in sys.path:
    sys.path.insert(0, '../scripts')
import cvsupport
import streams_operations
import streams_render
import streamsx.eventstreams as eventstreams
from streamsx.topology.schema import CommonSchema

from IPython.core.display import HTML

## has credentials for cloud & kafka/event stream
import credential


import logging
logging.getLogger("imgAna_OpenCV")



In [ ]:
## HACK - his should note be necessary - something is wrong with my environment, 
##
import os
try:
    print("JAVA_HOME is set to",os.environ["JAVA_HOME"])
except KeyError as err:
    os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_221.jdk/Contents/Home"
    print("Had to set JAVA_HOME is set to",os.environ["JAVA_HOME"],"\n   --- what is going on here...")


In [ ]:
def catchInterrupt(func):
    """decorator : when interupt occurs the display is lost if you don't catch it
       TODO * <view>.stop_data_fetch()  # stop
       
    """
    def catch_interrupt(*args, **kwargs):
        try: 
            func(*args, **kwargs)
        except (KeyboardInterrupt): pass
    return catch_interrupt

## Render with face_rects[] - reduce dependency on numpy
Using image from web add the rects. 

- Render in a widget, in a browser. 



In [ ]:
def line_box(ele):
    (x,y,w,h)=ele
    return (x,y, x+w,y, x+w,y+h, x,y+h, x,y)
    

def inscribe_rect(bin_image, detection_box, box_line_width=10):
    """Inscribe box on image
    
    This is updating the image passed in.
    
    Args:
        bin_image : binary image
        detection_box : region to put box around
    Return:
        return image - 
    """
    draw = ImageDraw.Draw(bin_image) 
    draw.line(line_box(detection_box), fill="yellow", width=box_line_width)
    return bin_image
   

def encode_img(img):
    """must be easier way"""
    with io.BytesIO() as output:
        img.save(output, format="JPEG")
        contents = output.getvalue() 
    return base64.b64encode(contents).decode('ascii')

def decode_img(bin64):
    """must be easier way"""
    img = Image.open(io.BytesIO(base64.b64decode(bin64)))
    return img

def resize_image(bin_image, basewidth=None, baseheight=None):
    """Resize image proportional to the base, make it fit in cell"""
    if basewidth is not None:
        wpercent = (basewidth/float(bin_image.size[0]))
        hsize = int((float(bin_image.size[1])*float(wpercent)))
        return bin_image.resize((basewidth,hsize), Image.ANTIALIAS)
    wpercent = (baseheight/float(bin_image.size[1]))
    wsize = int((float(bin_image.size[0])*float(wpercent)))
    return bin_image.resize((wsize,baseheight), Image.ANTIALIAS)

<a name="setup"></a>
# Setup
### Add credentials for the IBM Streams service

#### ICPD setup

With the cell below selected, click the "Connect to instance" button in the toolbar to insert the credentials for the service.

<a target="blank" href="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/02/connect_icp4d.gif">See an example</a>.

In [ ]:

# If you using ICP4D insert your creds here...
#from icpd_core import icpd_util
#global cfg
#cfg=icpd_util.get_service_instance_details(name='zen-sample-icp1-blitz-env')

# Accessing Streams Instance & views

instance,cfg = streams_operations.get_instance(service_name='Streaming3Turbine')

In [ ]:
# TODO move to commonStreams
# TODO kill the process if it is running before submitted.
def commonSubmit(_cfg, streams_cloud_service, _topo):
    """submit to either the cloud or CP4D
    Args:
        _cfg: when submitting from CP4D.
        streams_cloud_service : when submitting to cloud, the name of the service credential.py must appropriate mapping
        _topo : topology to submit
    
    """
    if _cfg is not None:
        # Disable SSL certificate verification if necessary
        _cfg[context.ConfigParams.SSL_VERIFY] = False
        submission_result = context.submit("DISTRIBUTED",_topo, config=_cfg)
    if _cfg is None:
        cloud = {
            context.ConfigParams.VCAP_SERVICES: credential.vcap_conf,
            context.ConfigParams.SERVICE_NAME: streams_cloud_service,
            context.ContextTypes.STREAMING_ANALYTICS_SERVICE:"STREAMING_ANALYTIC",
            context.ConfigParams.FORCE_REMOTE_BUILD: True,
        }
        submission_result = context.submit("STREAMING_ANALYTICS_SERVICE",_topo,config=cloud)

    # The submission_result object contains information about the running application, or job
    if submission_result.job:
        report = "JobId:{} Name:{} ".format(submission_result['id'], submission_result['name'])
    else:
        report = "Somthing did work:{}".format(submission_result)


In [ ]:
## Move toe commonDisplay.py

def display_image(view_tuple, image_region=None, rect_region=None, title_region=None, status_region=None, url_region=None):
    img_url = view_tuple['img_desc'][0]['img']
    response = requests.get(img_url)
    
    img_raw = bts_to_img(response.content)
    if img_raw is None:
        print("img_url:{} bts_img() failed".format(img_url))
        return
    img_rgb = convertToRGB(img_raw)
    face_rects = detect_faces(img_rgb, haar_cascade_face)
    status_region.value = "Found: {} potential faces. \n {}".format(len(face_rects),  face_rects)
    # img_rects = rects_render(img_rgb, face_rects)
    url_region.value = img_url
    title_region.value = view_tuple['title']
    with Image.open(io.BytesIO(response.content)) as bin_image:
        with image_region:
            display(resize_image(bin_image, baseheight=300))
            clear_output(wait=True)
            for rect in face_rects:
                inscribe_rect(bin_image, rect)
            with rect_region:
                if len(face_rects) is not 0:
                    display(resize_image(bin_image, baseheight=300))
                    clear_output(wait=True)
                else:
                    clear_output()
         

##   To Processing Operators. 
Above we created code to fetch a image from the web and find faces, which is composed a number of functions. 
To push it up to Streams we've rolled up the processing into two functions.

The cvsupport.py in [scripts](../scripts) class that are used below to compose the application.
- ImageFetch[] - web request that puts image in tuple. The most time consuming portion of the processing is fetching the images using the URL. Retrieve the image from and push the image data into the Stream where it will be processed by downstream operator.
- FaceRegions[] - locates faces in image 

# Compose and Submit the applications.

The Streams application that will get Tuples by subscribing a feed
that is composed in the [WikiRecentPhase3](./imgAna_3.jupyter-py36.ipynb).

### what applications are currently running....
Cancel those that you are about to submit...

In [ ]:
streams_render.list_jobs(instance, cancel=True)

### What views are available?
At the start only WikiPhase3


In [ ]:
streams_render.display_views(instance, "VideoRcvKafka")
streams_render.display_views(instance, "ImageAnalysis")

## Compose and submit 'VideoRcvKafka' 
VideSndKafka.py opens video files and sends frames via EventStream/Kafka. The received frames
are published to 'image_active'.



In [ ]:
def VideoRcvKafka():
    """Recieve video frame on topic and publish to 'image_string'. 
    
    Notes:
        - The script VideoSndKafka.py pushed video frame onto the topic.
    
    
    """
    topo = Topology("VideoRcvKafka")
    topo.add_pip_package('opencv-contrib-python')

    video_chunk = eventstreams.subscribe(topo, schema=CommonSchema.Json, 
                                         credentials=json.loads(credential.magsEventStream),
                                         topic='VideoFrame' )
    kafka_frame = video_chunk.map(cvsupport.BuildVideoFrame(), name="kafka_frame")
    kafka_frame.view(name="frame_kafka", description="frame from kafka image")
    kafka_frame.publish(topic="image_active")
    return topo


commonSubmit(cfg, "Streaming3Turbine", VideoRcvKafka())  

### Is 'VideoRcvKafka' view up.
The video feed frames are sent by VideoSndKafka.py, via EventStream/Kafka to this application.
The received frames are published.


In [ ]:
streams_render.display_views(instance, job_name="VideoRcvKafka")

## Compose and submit 'ImageAnalysis' application
- Two analysis operators thatdo analysis: findFace , findObject.
- Two applications feeding via pub/sub : VideoRcvKafka, ImageWebFetch
- This takes a longgg time to subumit...


In [ ]:
## compose the findImageElements application
def ImageAnalysis():
    """Analyse the images with various tools. 
       - Subscribe to 'image_active' with encoded images in 'image_string'
       - process 'image_string' with FaceRegions into 'face_regions' 
       - process 'image_string' with ObjectRegions into 'object_regions'
       - 
       
    """
    topo = Topology("ImageAnalysis")
    topo.add_file_dependency('../datasets/haarcascade_frontalface_default.xml', 'etc')
    topo.add_file_dependency('../datasets/yolov3.weights', 'etc')
    topo.add_file_dependency('../datasets/yolov3.cfg', 'etc')
    topo.add_pip_package('opencv-contrib-python')
    
    image_active = topo.subscribe(topic="image_active")
    # Find faces analysis ....
    face_regions = image_active.map(cvsupport.FaceRegions(), name="face_regions")
    face_trimmed = face_regions.map(lambda t: {
        'url':t['img_desc'][0]['img'], 
        'face_regions':t['face_regions'],
        'image_string':t['image_string']
    }, name="faces_trimmed")
    face_trimmed.view(name="faces_view", description="faces regions")
    # Find objects analysis ...
    object_regions = image_active.map(cvsupport.ObjectRegions(classes="../datasets/yolov3.txt"), name="object_fetch")
    object_trimmed = object_regions.map(lambda t: {
        'url':t['img_desc'][0]['img'], 
        'object_regions':t['object_regions'],
        'image_string':t['image_string']
    }, name="objects_trimmed")
    object_trimmed.view(name="objects_view", description="object regions")
    return topo


## TODO turn into function - huge duplication 
## Compose and submit the findElements Application 
topo = ImageAnalysis()


commonSubmit(cfg, "Streaming3Turbine", ImageAnalysis())    


# View the ImageAnalysis processing

All the processes 'healthy' ?

In [ ]:
streams_render.list_jobs(instance, cancel=True)

What are the ImageAnalysis views.

In [ ]:
streams_render.display_views(instance, job_name="ImageAnalysis")

## View the ImageAnalysis application's FacesRegions results

#### TODO - convert to thread

In [ ]:
## Fetch FacesRegions start/get/stop
faces_view = instance.get_views(name="faces_view")[0]
faces_view.start_data_fetch()
faces_tuples = faces_view.fetch_tuples(max_tuples=100, timeout=10)
print("Number of faces_tuples:", len(faces_tuples))
faces_view.stop_data_fetch()

In [ ]:
# render the results.... only images with regions will arrive via this view
source_face = widgets.Label(value="Source", layout={'border': '1px solid green','width':'50%'})
rect_face = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
url_face = widgets.Label(value="URL", layout={'border': '1px solid green','width':'50%'})
dashboard_face = widgets.VBox([source_face, rect_face, url_face])
display(dashboard_face)

# Fetch the tuples from the view
count = 0
for face in faces_tuples:
    url_face.value = face['url']
    #source_face.value = trimmed['source']  # TODO - move accros the source.
    source_face.value = str(count)
    count += 1
    image_string = face['image_string']
    with Image.open(io.BytesIO(base64.b64decode(image_string))) as bin_image:
        for rect in face['face_regions']:
            inscribe_rect(bin_image, rect)
        with rect_face:
            display(resize_image(bin_image, baseheight=600))
            clear_output(wait=True)
    time.sleep(2)


## View the ImageAnalysis application's ObjectRegions results
Display the images and the objects they found. Run in a thread to make is easier to stop.

On the Streams side it's processing the images to find objects, if not no objects are 
found they are not pushed to the view and will not be rendered here.

The object detection maybe CPU intensive (you don't say) you may want to distribute this across hardware, which
is left as an exercise to the user.



In [ ]:
import threading
import time
class objectDashboard(object):
    def __init__(self, instance, sleep=2):
        self.instance = instance
        self.source = widgets.Label(value="Source", layout={'border': '1px solid green','width':'50%'})
        self.rect = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
        self.url = widgets.Label(value="URL", layout={'border': '1px solid green','width':'50%'})
        self._class = widgets.Label(value="CLASS", layout={'border': '1px solid green','width':'50%'})
        self.button = widgets.Button(description='', button_style='danger', layout={'width':'25%'}, disabled=True)
        self.dashboard = widgets.VBox([self.source, self.rect, self.url, self.button, self._class])
        self.time_sleep = sleep
        self.execute = threading.Event()
        self.thread = None
        

    def render_tuples(self, tuples):
        count = 0
        self.source.value = "view dequed:{} @ {}".format(len(tuples), time.strftime("%H:%M:%S", time.localtime()))
        for tuple_object in tuples:
            self.url.value = tuple_object['url']
            self.source.value = "{} of {}".format(count, len(tuples))
            count += 1
            image_string = tuple_object['image_string']
            class_string = ""
            with Image.open(io.BytesIO(base64.b64decode(image_string))) as bin_image:
                for rect in tuple_object['object_regions']:
                    inscribe_rect(bin_image, rect['region'])
                    class_string = ",".join([class_string,rect['class']])
                    self._class.value = class_string
                with self.rect:
                    display(resize_image(bin_image, baseheight=600))
                    clear_output(wait=True)
            if not self.execute.is_set():
                break
            time.sleep(self.time_sleep)
            
    def ignition(self, start):
        if start:
            self.thread = threading.Thread(target=self.fetchRender_thread, name="Render_View")
            self.thread.start()
        else:
            self.execute.clear()
            self.button.description = "Stopping..."
            self.button.disabled = True
                

    def fetchRender_thread(self):
        self.button.disabled = True
        self.button.description = 'Stop'
        self.button.button_style = 'danger'
        button_action = lambda w: self.ignition(False)
        self.button.on_click(button_action)
        self.button.disabled = False
        self.execute.set()

        # start up
        objects_view = self.instance.get_views(name="objects_view")[0]
        objects_view.start_data_fetch()
        # fetch til button pushed
        while self.execute.is_set():
            objects_tuples = objects_view.fetch_tuples(max_tuples=100, timeout=2)
            self.render_tuples(objects_tuples)
        # shut down
        objects_view.stop_data_fetch()
        # 
        self.button.description = 'Stopped'
        self.button.button_style = 'warning'

        
object_dash = objectDashboard(instance, sleep=0.1)
display(object_dash.dashboard)
object_dash.ignition(True)



In [ ]:
object_dash.execute.clear()
